In [1]:
from sklearn.cluster import KMeans
import sklearn.cluster
import numpy as np
import pandas as pd
from scipy.spatial.distance import squareform, pdist
from scipy.spatial import distance
from scipy.spatial import distance_matrix
import re
import itertools

import random

## Define a function for calculating clusters of mutations

In [2]:
def clustergroupings(l):
    out = []
    while len(l)>0:
        first, rest = l[0], l[1:]
        first = set(first)

        lf = -1
        while len(first)>lf:
            lf = len(first)

            rest2 = []
            for r in rest:
                if len(first.intersection(set(r)))>0:
                    first |= set(r)
                else:
                    rest2.append(r)     
            rest = rest2

        out.append(first)
        l = rest

    return out

## We load a structure of NALCN and a file containing the lists of mutations

In [3]:
pdb = open("./NALCN_WT.pdb")
pdb_lines = pdb.readlines()
pdb.close()

In [4]:
f = open("./NALCN_GImutations_summed.csv", "r")
mutants = f.readlines()[1:]
f.close()

## We go through the pdb file and extract the coordinates for the mutated residues

In [5]:
mutationmatrix = []
for pdbline in pdb_lines:
    if pdbline.startswith("ATOM"):
        if pdbline.split()[2] == "CA":
            residue_number = int(pdbline[22:26])
            xcoord = float(pdbline.split()[6])
            ycoord = float(pdbline.split()[7])
            zcoord = float(pdbline.split()[8])
            chain = pdbline.split()[4]
            aa = pdbline.split()[3]
            matchcounter=0
            for line in mutants:
                index, aminoacid, mutationtype, mutationcount = line.split(",")
                if int(aminoacid) == residue_number:
                    if mutationtype != "FS":
                        matchcounter +=1
                        count = int(mutationcount)
                        mutationmatrix.append([xcoord,ycoord,zcoord,residue_number,count,chain, aa])
            if matchcounter == 0:
                mutationmatrix.append([xcoord,ycoord,zcoord,residue_number,0, chain, aa])

                
## Make a dataframe of the resultant array 
mutationdataframe = pd.DataFrame(mutationmatrix, columns=["X", "Y", "Z", "Resid", "Count", "Chain", "AA"])

## Subset the dataframe into mutated residues only
mutationdataframe_only = mutationdataframe.loc[mutationdataframe['Count'] != 0]

## Get only chain A (the NALCN structure without FAM155C)
mutationdataframe_only_chainA = mutationdataframe_only[mutationdataframe_only["Chain"] == "A"]

## Convery X, Y, and Z into floats
mutationdataframe_only_chainA["X"] = pd.to_numeric(mutationdataframe_only_chainA["X"], errors='coerce').fillna(0)
mutationdataframe_only_chainA["Y"] = pd.to_numeric(mutationdataframe_only_chainA["Y"], errors='coerce').fillna(0)
mutationdataframe_only_chainA["Z"] = pd.to_numeric(mutationdataframe_only_chainA["Z"], errors='coerce').fillna(0)

C:\Users\ds728\anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\ds728\anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\ds728\anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

## Generate a set of mutational clusters within a set distance (10 Angstroms)

In [6]:
cutoff = 10
# calculate pairwise distance matrix
pair_dist = distance_matrix(mutationdataframe_only_chainA[["X","Y","Z"]], mutationdataframe_only_chainA[["X","Y","Z"]])
# assign ids
ids = np.argwhere(pair_dist < cutoff)
# group the clusters together
mutationclusters = clustergroupings(ids)

In [7]:
## List the clusters of mutations
mutationclusters

[{0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  13,
  14,
  15,
  18,
  19,
  20,
  21,
  66,
  67,
  68,
  69,
  71,
  72},
 {12},
 {16, 17},
 {22, 23, 24, 50},
 {25, 47, 48, 49, 81},
 {26,
  27,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  73,
  74,
  75,
  76},
 {28},
 {29, 30, 31, 32},
 {51, 52, 53, 54},
 {55, 56, 57},
 {58},
 {59, 60, 61, 62, 63},
 {64},
 {65},
 {70},
 {77, 78, 79, 80},
 {82},
 {83},
 {84, 85, 86},
 {87, 88, 89, 90, 95, 96},
 {91},
 {92, 93, 94}]

## Define the biggest clusters with this distance

In [8]:
## Big cluster 1
mutationdataframe_only_chainA.iloc[list(mutationclusters[0])]

,X,Y,Z,Resid,Count,Chain,AA
6,151.710,189.203,155.527,39,1,A,HIS
10,152.059,184.619,151.304,43,5,A,ARG
17,157.285,178.458,144.034,50,1,A,VAL
18,155.570,179.212,140.713,51,1,A,ILE
19,158.736,180.894,139.432,52,1,A,SER
20,161.133,178.142,140.501,53,1,A,VAL
22,159.227,177.501,135.633,55,4,A,MET
24,163.747,175.019,133.524,57,1,A,THR
453,156.855,171.229,135.350,540,1,A,PHE
452,159.082,170.671,132.345,539,1,A,THR


In [9]:
voltage_sensor_mutations = mutationdataframe_only_chainA.iloc[list(mutationclusters[0])]["Resid"].tolist()
len(voltage_sensor_mutations)

25

In [10]:
## Big cluster 2
mutationdataframe_only_chainA.iloc[list(mutationclusters[5])]

,X,Y,Z,Resid,Count,Chain,AA
462,151.513,158.657,138.292,549,1,A,GLN
465,148.293,156.422,141.592,552,2,A,THR
466,150.768,153.470,141.641,553,1,A,GLN
468,153.445,152.303,136.369,555,1,A,GLY
145,163.362,138.777,150.139,193,1,A,GLY
156,168.299,135.268,132.617,204,1,A,THR
190,160.482,124.090,127.561,238,1,A,GLN
197,167.316,127.956,126.318,245,1,A,CYS
205,174.668,141.361,128.206,253,1,A,LEU
209,169.735,145.762,129.339,257,1,A,GLU


In [11]:
pore_mutations = mutationdataframe_only_chainA.iloc[list(mutationclusters[5])]["Resid"].tolist()
len(pore_mutations)

20

# Start the permutation test

## Define a function for calculating the mutational clusters with a given cutoff, and a given set of mutations

In [12]:
def calculatemutationclusters(mutationlist, pdbframe, cutoff):
    mutationsonly = pdbframe[pdbframe["Resid"].isin(mutationlist)]
    pair_dist = (distance_matrix(mutationsonly[["X","Y","Z"]], mutationsonly[["X","Y","Z"]]))
    #pair_dist.values[np.triu_indices_from(pair_dist, 0)] = np.nan
    ids = np.argwhere(pair_dist < cutoff)
    mutationclusters = clustergroupings(ids)
    lengths = [len(x) for x in mutationclusters]
    return(lengths)

In [13]:
mutationdataframe_chainA = mutationdataframe[mutationdataframe["Chain"] == "A"]

In [14]:
mutationdataframe_chainA

,X,Y,Z,Resid,Count,Chain,AA
0,156.097,194.978,159.602,33,0,A,ILE
1,153.175,196.400,161.596,34,0,A,ASN
2,151.042,196.509,158.444,35,0,A,LYS
3,148.597,193.584,158.072,36,0,A,PRO
4,148.686,193.619,154.268,37,0,A,TRP
...,...,...,...,...,...,...,...
664,124.305,127.113,148.486,995,3,A,ARG
665,127.111,127.210,151.059,996,0,A,ILE
666,126.041,130.769,151.892,997,0,A,PHE
667,122.620,129.472,152.959,998,0,A,LYS


In [15]:
nperm = 100000

clusters_array = []
for i in range(0,nperm):
    if i % 10000 == 0:
        print(i)
    # generate sample of same length as number of mutations
    sample = random.sample(range(0,len(mutationdataframe_chainA)),len(mutationdataframe_only_chainA))
    residues = mutationdataframe_chainA.iloc[sample]["Resid"].tolist()
    # calculate all of the clusters
    clusters = calculatemutationclusters(residues, mutationdataframe_chainA, 10)
    clusters_array.append(clusters)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000


## Cluster 1 p-value

In [16]:
count1 = 0
for item in clusters_array:
    if any(x >= len(voltage_sensor_mutations) for x in item):
        count1+=1

In [17]:
print(count1/nperm)

0.28916


## Cluster 2 p-value

In [18]:
count2 = 0
for item in clusters_array:
    if any(x >= len(pore_mutations) for x in item):
        count2+=1

In [19]:
print(count2/nperm)

0.55414


## Probability of seeing 2 clusters of 20 and 25 residues

In [29]:
count3 = 0
for item in clusters_array:
    x = [i for i in item if i >= len(pore_mutations)]
    if len(x) >=2:
        if any(x2 >=len(voltage_sensor_mutations) for x2 in x):
            count3+=1

In [30]:
print(count3/nperm)

0.02725
